In [1]:
import numpy as np
import util
import pandas as pd

In [13]:
class MagneticMirror2D(object):

    #######################################################################
    # Begin physical parameters                                           #
    #######################################################################

    # domain size in meter
    Lx = 1e-2 # r direction
    Lz = 1e-1

    # spatial resolution in number of cells
    Nx = 32
    Nz = 256

    # mirror ratio
    R = 10.0
    B_max = 30.0  # T

    # use a reduced ion mass for faster simulations
    m_ion = 400.0 * util.constants.m_e
    # m_ion = util.constants.m_p

    # initial seed macroparticle density
    nppc_seed = 5  # 800

    # total simulation time in ion thermal crossing times
    crossing_times = 1

    def __init__(self):
        # plasma density m^{-3}
        self.n0 = 1e16

        # temperature
        self.Te = 400 # eV
        self.Ti = self.Te

        self.dx = self.Lx / self.Nx
        self.dz = self.Lz / self.Nz

        # calculate electron plasma frequency
        w_pe = util.plasma_freq(self.n0)
        self.dt = 0.07 / w_pe
        # simulation timestep from electron CFL
        # self.dt = self.dz / (5.0 * util.thermal_velocity(self.Te, util.constants.m_e))

        # calculate the ion crossing time to get the total simulation time
        ion_crossing_time = self.Lz / \
            util.thermal_velocity(self.Ti, self.m_ion)
        self.total_steps = int(np.ceil(
            self.crossing_times * ion_crossing_time / self.dt
        ))
        self.diag_steps = int(self.total_steps / 20.0)

        # calculate the flux from the thermal plasma reservoir
        self.flux_e = (
            self.n0 * util.thermal_velocity(self.Te, util.constants.m_e) / np.sqrt(2.0 * np.pi)
        )
        # self.flux_i = self.flux_e * np.sqrt(util.constants.m_e / self.m_ion)
        self.flux_i = (
            self.n0 * util.thermal_velocity(self.Ti, self.m_ion)
        )

        # check spatial resolution
        self.debye_length = util.debye_length(self.Te, self.n0)

        print("Starting simulation with parameters:")
        print(f"    Lz = {self.Lz}m, Lx = {self.Lx}m")
        print(f"    Nz = {self.Nz}, Nx = {self.Nx}")
        print(f"    dz = {self.dz*1e6:.1f}um, dx = {self.dx*1e6:.1f}um")
        print(f"    Debye length = {self.debye_length*1e6:.1f} um")
        print(f"    Electron plasma frequency = {w_pe:.1f} Hz")
        print(f"    T_e = T_i = {self.Te:.3f} eV")
        print(f"    n0 = {self.n0:.1e} m^-3")
        print(f"    M/m = {self.m_ion/util.constants.m_e:.0f}")
        # print(f"    flux_e = {self.flux_e*util.constants.e:.1f} A/m2")
        print(f"    flux_i = {self.flux_i*util.constants.e:.1f} A/m2")
        print(f"    Ion crossing time: {ion_crossing_time}")
        print(f"    Ion crossing steps: {ion_crossing_time / self.dt:.0f}")
        print(f"    Total steps = {self.total_steps}")
mirror = MagneticMirror2D()


Starting simulation with parameters:
    Lz = 0.1m, Lx = 0.01m
    Nz = 256, Nx = 32
    dz = 390.6um, dx = 312.5um
    Debye length = 1486.0 um
    Electron plasma frequency = 5641460231.2 Hz
    T_e = T_i = 400.000 eV
    n0 = 1.0e+16 m^-3
    M/m = 400
    flux_i = 950.2 A/m2
    Ion crossing time: 1.686131268092182e-07
    Ion crossing steps: 13589
    Total steps = 13589


In [14]:
num_cores = 2**np.arange(8)
time_per_step_growing_rate = (1 - 0.03)/1000 # in 1000 steps, it grows from 0.03 to 1
time_per_step = np.linspace(0.03, mirror.total_steps*time_per_step_growing_rate, mirror.total_steps) 
total_time = np.round(time_per_step.sum() / num_cores / 3600, 1) # in hours
pd.DataFrame({"Cores": num_cores, "Total Hours": total_time, "Total Days": np.round(total_time/24,1)})

,Cores,Total Hours,Total Days
0,1,24.9,1.0
1,2,12.5,0.5
2,4,6.2,0.3
3,8,3.1,0.1
4,16,1.6,0.1
5,32,0.8,0.0
6,64,0.4,0.0
7,128,0.2,0.0
